In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn 
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.1.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split
x_train_all,x_test,y_train_all,y_test=train_test_split(housing.data,housing.target,random_state=7)
x_train,x_valid,y_train,y_valid=train_test_split(x_train_all,y_train_all,random_state=11)

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(3870, 8) (3870,)
(11610, 8) (11610,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
x_train_s=scaler.fit_transform(x_train)
x_valid_s=scaler.fit_transform(x_valid)
x_test_s=scaler.transform(x_test)

In [7]:
# metric 使用

metric=keras.metrics.MeanSquaredError()
print(metric([5.],[2.]))
print(metric([0.],[2.]))
print(metric.result())

metric.reset_states()
metric([1.],[3.])
print(metric.result())

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.5, shape=(), dtype=float32)
tf.Tensor(6.5, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [8]:
# 1. batch 遍历训练集 metric
#    1.1 自动求导
# 2. epoch 结束  验证集 metric

epochs=100
bs=32
steps_per_epoch=len(x_train_s)//bs
optimizer=keras.optimizers.SGD()
metric=keras.metrics.MeanSquaredError()

def random_batch(x,y,bs=32):
    idx = np.random.randint(0,len(x),size=bs)
    return x[idx],y[idx]

model=keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])
for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_b,y_b=random_batch(x_train_s,y_train,bs)
        
        with tf.GradientTape() as tape:
            y_pred=model(x_b)
            loss = tf. reduce_mean(keras.losses.mean_squared_error(y_b,y_pred))
            metric(y_b,y_pred)
        grads=tape.gradient(loss,model.variables)
        grads_and_vars=zip(grads,model.variables)
        print('\rEpoch',epoch,'train mse:',metric.result().numpy(),end='')
    y_v_pred=model(x_valid_s)
    valid_loss=tf.reduce_mean(keras.losses.mean_squared_error(y_v_pred,y_valid))
    print('\t','valid mse:',valid_loss.numpy())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0 train mse: 7.4651025 0 train mse: 7.2201987.2897363train mse: 7.484408	 valid mse: 7.514695313928965
Epoch 1 train mse: 7.5499444 7.477252 1 train mse: 7.4036956	 valid mse: 7.514695313928965
Epoch 2 train mse: 7.3895802	 valid mse: 7.514695313928965se: 7.4180713
Epoch 3 train mse: 7.57941753 train mse: 6.853963 3 train mse: 8.89248 7.9041557 7.731439	 valid mse: 7.514695313928965
Epoch 4 train mse: 7.451952	 valid mse: 7.514695313928965se: 7.454432
Epoch 5 train mse: 7.640652	 valid mse: 7.5146

Epoch 85 train mse: 7.2216206 85 train mse: 7.289991 85 train mse: 7.1352916 85 train mse: 7.251398	 valid mse: 7.514695313928965
Epoch 86 train mse: 8.0255045	 valid mse: 7.514695313928965
Epoch 87 train mse: 7.5269096 87 train mse: 7.5768485	 valid mse: 7.514695313928965
Epoch 88 train mse: 7.3606873	 valid mse: 7.514695313928965in mse: 7.418545 88 train mse: 7.409473
Epoch 89 train mse: 7.2360385 89 train mse: 7.1223116	 valid mse: 7.514695313928965
Epoch 90 train mse: 7.2731075train mse: 7.4172335 90 train mse: 7.2586503 90 train mse: 7.2838073	 valid mse: 7.514695313928965
Epoch 91 train mse: 7.2240456train mse: 7.1541533	 valid mse: 7.514695313928965
Epoch 92 train mse: 7.544037	 valid mse: 7.514695313928965
Epoch 93 train mse: 7.3761706 93 train mse: 7.49835 93 train mse: 7.378527	 valid mse: 7.514695313928965
Epoch 94 train mse: 7.2519293	 valid mse: 7.514695313928965
Epoch 95 train mse: 7.3555603	 valid mse: 7.514695313928965
Epoch 96 train mse: 7.3740196	 valid mse: 7.5146953

In [ ]:
history=model.fit(x_train_s,y_train,validation_data=(x_valid_s,y_valid),epochs=100,callbacks=callbacks)

In [9]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
    
plot_learning_curves(history)

NameError: name 'history' is not defined

In [ ]:
model.evaluate(x_test_s,y_test)